In [208]:
import polars as pl
import pandas as pd

In [210]:
df = pd.read_csv('1,2M_Tracks_with_playlists.csv')

C:\Users\juan\AppData\Local\Temp\ipykernel_17128\1624177740.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('1,2M_Tracks_with_playlists.csv')


In [211]:
# Filter out playlists with less than 10 tracks, not null and nan
df_playlsits = df[df["playlist_count"].notnull()]

df_playlsits = df_playlsits[df_playlsits["playlist_count"] >= 10]
df_playlsits = df_playlsits.dropna()

In [212]:
# Limit of columns width
pl.Config.set_fmt_str_lengths(100)

polars.cfg.Config

In [213]:
# # Remove all '[', ']' and '"'"
df_playlsits["playlist_id"] = df_playlsits["playlist_id"].str.replace("[", "")
df_playlsits["playlist_id"] = df_playlsits["playlist_id"].str.replace("]", "")
df_playlsits["playlist_id"] = df_playlsits["playlist_id"].str.replace("'", "") 
df_playlsits["playlist_id"] = df_playlsits["playlist_id"].str.replace(" ", "") 

# Split the column
df_playlsits["playlist_id"] = df_playlsits["playlist_id"].str.split(",")


C:\Users\juan\AppData\Local\Temp\ipykernel_17128\3830364219.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_playlsits["playlist_id"] = df_playlsits["playlist_id"].str.replace("[", "")
C:\Users\juan\AppData\Local\Temp\ipykernel_17128\3830364219.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_playlsits["playlist_id"] = df_playlsits["playlist_id"].str.replace("]", "")


In [246]:
# Get all the unique playlists from the column playlist_id
unique_playlists = []
for i in df_playlsits["playlist_id"]:
    for j in i:
        if j == "nan":
            i.remove(j)
        else:
            j = int(j)
            if j not in unique_playlists:
                unique_playlists.append(j)

In [248]:
print(len(unique_playlists))

5000


In [299]:
import numpy as np
from datasketch import MinHash, MinHashLSHForest
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
# SpectralClustering
from sklearn.cluster import SpectralClustering

In [266]:
# Load the dataset
song_data = (df_playlsits[["track_uri", "playlist_id"]].values)

# Create a dictionary of playlists for each song
song_playlists = {}
for song_id, playlist_ids in song_data:
    song_playlists[song_id] = set(playlist_ids)

In [267]:
# Define the number of hash functions and bands
n_hash_functions = 128
n_bands = 64

# Create MinHash signatures for each song
minhashes = {}
for song_id, playlists in song_playlists.items():
    mh = MinHash(num_perm=n_hash_functions)
    for playlist in playlists:
        mh.update(str(playlist).encode('utf-8'))
    minhashes[song_id] = mh

In [271]:
# Create a MinHash LSH forest and index the signatures
forest = MinHashLSHForest(num_perm=n_hash_functions)
for song_id, mh in minhashes.items():
    forest.add(song_id, mh)
forest.index()

In [272]:
# Compute the Jaccard similarity between all pairs of songs
similarity_matrix = np.zeros((len(song_data), len(song_data)))
for i, (song_id1, _) in enumerate(song_data):
    mh1 = minhashes[song_id1]
    for j, (song_id2, _) in enumerate(song_data[i+1:], i+1):
        mh2 = minhashes[song_id2]
        similarity_matrix[i, j] = mh1.jaccard(mh2)
        similarity_matrix[j, i] = similarity_matrix[i, j]

In [302]:
# Cluster the songs using spectral clustering
clustering = SpectralClustering(n_clusters=100, affinity='precomputed', n_init=100, n_jobs=-1)
clustering.fit(similarity_matrix)

-0.04299860150244076

In [303]:
# Assign each song to a cluster
song_clusters = {}
for i, (song_id, _) in enumerate(song_data):
    song_clusters[song_id] = clustering.labels_[i]
# Count the number of songs in each cluster
cluster_counts = {}
for song_id, cluster_id in song_clusters.items():
    if cluster_id not in cluster_counts:
        cluster_counts[cluster_id] = 0
    cluster_counts[cluster_id] += 1

print(cluster_counts)

{33: 7986, 40: 628, 91: 163, 93: 283, 99: 319, 62: 416, 46: 181, 1: 279, 60: 216, 72: 207, 69: 2644, 95: 177, 15: 44, 53: 317, 67: 82, 27: 217, 0: 237, 64: 80, 97: 114, 50: 138, 71: 232, 47: 71, 79: 120, 88: 175, 59: 51, 42: 68, 19: 104, 57: 70, 61: 45, 55: 149, 28: 171, 58: 92, 70: 44, 86: 83, 21: 188, 41: 76, 63: 100, 75: 53, 82: 71, 32: 35, 89: 57, 68: 66, 77: 13, 96: 48, 83: 50, 39: 115, 2: 57, 87: 23, 81: 39, 9: 29, 38: 54, 35: 61, 44: 35, 52: 24, 16: 18, 85: 75, 18: 54, 26: 38, 25: 32, 37: 70, 80: 26, 98: 33, 29: 30, 76: 41, 4: 18, 90: 83, 36: 32, 17: 62, 78: 65, 51: 16, 30: 36, 20: 62, 45: 52, 94: 17, 65: 39, 10: 24, 6: 27, 84: 54, 74: 51, 49: 38, 24: 18, 54: 11, 48: 20, 43: 20, 13: 11, 7: 12, 3: 9, 8: 14, 56: 9, 92: 15, 23: 7, 5: 8, 34: 23, 11: 16, 22: 37, 66: 14, 73: 5, 31: 5, 12: 6, 14: 10}


In [275]:
# Create a DataFrame with the song clusters
song_clusters = pd.DataFrame({'song_id': [song[0] for song in song_data], 'cluster': clustering.labels_})

# Save the DataFrame to a CSV file
song_clusters.to_csv('song_clusters.csv', index=False)